In [4]:
%pip install azure-cognitiveservices-speech --quiet
%pip install openai==1.30.1 --quiet
%pip install sounddevice --quiet
%pip install azure-ai-formrecognizer azure-cognitiveservices-speech azure-identity --quiet
%pip install load_dotenv --quiet
%pip install prompt_toolkit --quiet
%pip install azure-storage-blob --quiet
%pip install neo4j --quiet


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('.env')

# Access the environment variables
speech_key = os.getenv('SPEECH_KEY')
speech_region = os.getenv('SPEECH_REGION')
whisper_key = os.getenv('WHISPER_KEY')
whisper_endpoint = os.getenv('WHISPER_ENDPOINT')
whisper_deployment = os.getenv('WHISPER_DEPLOYMENT')
gpt_key = os.getenv('GPT_KEY')
gpt_deployment = os.getenv('GPT_DEPLOYMENT')
gpt_endpoint = os.getenv('GPT_ENDPOINT')
storage_endpoint = os.getenv('AZURE_STORAGE_ENDPOINT')
storage_sas = os.getenv('AZURE_STORAGE_SAS_TOKEN')

# Use the environment variables in your code
print(f'Speech API Key: {speech_key}')
print(f'Speech Region: {speech_region}')
print(f'Whisper API Key: {whisper_key}')
print(f'Whisper Endpoint: {whisper_endpoint}')
print(f'Whisper Deployment: {whisper_deployment}')
print(f'GPT API Key: {gpt_key}')
print(f'GPT Deployment: {gpt_deployment}')
print(f'GPT Endpoint: {gpt_endpoint}')
print(f'Storage Endpoint: {storage_endpoint}')
print(f'Storage SAS Token: {storage_sas}')

Speech API Key: f07276fad5414743b35c407402f0e57d
Speech Region: swedencentral
Whisper API Key: ad0371a819a94fafba0e78d596097a45
Whisper Endpoint: https://ut-oai-sweden.openai.azure.com/
Whisper Deployment: whisper
GPT API Key: 8873c265d70546a3a87dcb073310a16b
GPT Deployment: gpt-4o
GPT Endpoint: https://ut-oai-eastus.openai.azure.com/
Storage Endpoint: https://stutwenteai759783997450.blob.core.windows.net/
Storage SAS Token: sv=2022-11-02&ss=b&srt=co&sp=rl&se=2024-09-24T02:44:59Z&st=2024-06-02T18:44:59Z&spr=https&sig=5bc8o%2F1eSUUYSp5%2FwmrXESAdZJYz%2BqEIRkXaKnvE3gY%3D


In [11]:
import azure.cognitiveservices.speech as speechsdk

def recognize_from_microphone():
    # This example requires environment variables named "SPEECH_KEY" and "SPEECH_REGION"
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region)
    speech_config.speech_recognition_language="en-US"

    audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    print("Speak into your microphone.")
    speech_recognition_result = speech_recognizer.recognize_once_async().get()

    if speech_recognition_result.reason == speechsdk.ResultReason.RecognizedSpeech:
        #print("Recognized: {}".format(speech_recognition_result.text))
        return speech_recognition_result.text
    elif speech_recognition_result.reason == speechsdk.ResultReason.NoMatch:
        print("No speech could be recognized: {}".format(speech_recognition_result.no_match_details))
    elif speech_recognition_result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = speech_recognition_result.cancellation_details
        print("Speech Recognition canceled: {}".format(cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            print("Error details: {}".format(cancellation_details.error_details))
            print("Did you set the speech resource key and region values?")

text = recognize_from_microphone()
print(text)

Speak into your microphone.
Hello there.


In [ ]:
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key=whisper_key,
    api_version="2024-02-01",
    azure_endpoint = whisper_endpoint
)

audio_test_file = "./helloThere.m4a"

result = client.audio.transcriptions.create(
    file=open(audio_test_file, "rb"),            
    model=whisper_deployment
)

print(result)

In [17]:
'''
  For more samples please visit https://github.com/Azure-Samples/cognitive-services-speech-sdk 
'''

import azure.cognitiveservices.speech as speechsdk

speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region)
# Note: the voice setting will not overwrite the voice element in input SSML.
speech_config.speech_synthesis_voice_name = "en-GB-RyanNeural"

text = "Hi, this is Ryan"

def text_to_speech(text):
  # use the default speaker as audio output.
  speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config)
  result = speech_synthesizer.speak_text_async(text).get()
  # Check result
  if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
      print("Speech synthesized for text [{}]".format(text))
  elif result.reason == speechsdk.ResultReason.Canceled:
      cancellation_details = result.cancellation_details
      print("Speech synthesis canceled: {}".format(cancellation_details.reason))
      if cancellation_details.reason == speechsdk.CancellationReason.Error:
          print("Error details: {}".format(cancellation_details.error_details))

text_to_speech(text)



Speech synthesized for text [Hi, this is Ryan]


In [ ]:
import os
from azure.storage.blob import BlobServiceClient


# Create the BlobServiceClient object using the account URL and SAS token
blob_service_client = BlobServiceClient(account_url=storage_endpoint, credential=storage_sas)
blob_container_client = blob_service_client.get_container_client("paintings")
blob_list = blob_container_client.list_blobs()

for blob in blob_list:
    print(blob.name)

In [7]:
from neo4j import GraphDatabase


# Replace these with your Neo4j connection details
uri = "bolt://localhost:7687"  # Update this with your database URI
user = "neo4j"                 # Update this with your database username
password = "Pa$$w0rd"          # Update this with your database password

# Create a Neo4j driver instance
driver = GraphDatabase.driver(uri, auth=(user, password))

def get_cultural_heritage_node(name):
    with driver.session() as session:
        result = session.run("MATCH (n:CulturalHeritage {name: $name}) RETURN n", name=name)
        return [record["n"] for record in result]

# Query for a specific CulturalHeritage node by name
node_name = "King Caspar"
nodes = get_cultural_heritage_node(node_name)

# Extract the img URL and exhibit description
if nodes:
    node = nodes[0]  # Assuming you want the first node in the result
    properties = node._properties  # Access the properties of the node

    # Extract the `img` URL and `exhibit` description
    img_url = properties.get('img', 'No image URL found')
    exhibit_description = properties.get('exhibit', 'No description found')
    node_name = properties.get('name', 'No name found')

    print("Painting Name:", node_name)
    print("Image URL:", img_url)
    print("Description:", exhibit_description)
else:
    print("No nodes found")

# Close the driver connection
driver.close()

Node Name: King Caspar
Image URL: https://upload.wikimedia.org/wikipedia/commons/thumb/e/e5/Hendrick-Heerschop-Koning-Caspar-1654.-Olieverf-op-paneel.-Berlijn-Staatliche-Museen.jpg/485px-Hendrick-Heerschop-Koning-Caspar-1654.-Olieverf-op-paneel.-Berlijn-Staatliche-Museen.jpg
Description: Legend has it that one of the three magi who came to worship the Christ child was an African. Sometimes he is called Caspar, sometimes Balthasar. Heerschop painted him without surroundings or story. He can only be identified from his expensive clothes and the jar of incense he gave as his gift. But it is the man’s face that attracts the most attention; he looks at us proudly and self-confidently.


In [15]:
from promptflow.core import Prompty
from IPython.display import Image

image_path = img_url
print("Painting: " + image_path)
display(Image(url=image_path))
firstName = "John"
context = f"Painting Name: {node_name} Painting Information: {exhibit_description}"
question = "What can you tell me about the painting?"

def get_oai_response(firstName=firstName, context=context, question=question, image_path=image_path, conversation_history=[]):
    path_to_prompty = "basic.prompty"
    flow = Prompty.load(path_to_prompty)
    result = flow(
        firstName = firstName,
        context = context,
        question = question,
        image = image_path,
        conversation_history = conversation_history
    )

    return result

result = get_oai_response(firstName=firstName, question="What can you tell me about the painting?", image_path=image_path)
print(result)

Painting: https://upload.wikimedia.org/wikipedia/commons/thumb/e/e5/Hendrick-Heerschop-Koning-Caspar-1654.-Olieverf-op-paneel.-Berlijn-Staatliche-Museen.jpg/485px-Hendrick-Heerschop-Koning-Caspar-1654.-Olieverf-op-paneel.-Berlijn-Staatliche-Museen.jpg


Sure, John. The painting you're referring to is of King Caspar, created by Hendrick Heerschop in 1654. It's interesting because it depicts one of the magi, believed by legend to be an African king who came to worship the Christ child. Heerschop painted him without any background or narrative elements. The identity comes through his elaborate clothing and the jar of incense, his gift. However, the most captivating aspect is his face, which exudes pride and self-confidence.


In [ ]:
from IPython.display import Image, display

# URL of the image
image_url = "https://th.bing.com/th/id/R.842fb9a1885e50a762ef352821d9078d?rik=ykwt7kPK%2f3ngFA&riu=http%3a%2f%2fupload.wikimedia.org%2fwikipedia%2fcommons%2fe%2fe8%2fVan_Gogh_The_Olive_Trees..jpg&ehk=o8ZPcWcu3H0Vdk%2b2E5YJ63CXJSSHb3BFrzKL3UG5HOU%3d&risl=1&pid=ImgRaw&r=0"

# Display the image
display(Image(url=image_url))


In [18]:
conversation_history = []
image_path = img_url
print("Painting: " + image_path)
display(Image(url=image_path))
firstName = "Thomas"
context = f"Painting Name: {node_name} Painting Information: {exhibit_description}"

def dialog():
    text = recognize_from_microphone()
    result = get_oai_response(firstName=firstName, context=context, question=text, image_path=image_path, conversation_history=conversation_history)
    print(conversation_history)
    print(result)
    conversation_history.append(f'''
User: {text}
Assistant: {result}
''')
    speech = text_to_speech(result)
    if "goodbye" in text.lower():
        return
    dialog()
dialog()



Painting: https://upload.wikimedia.org/wikipedia/commons/thumb/e/e5/Hendrick-Heerschop-Koning-Caspar-1654.-Olieverf-op-paneel.-Berlijn-Staatliche-Museen.jpg/485px-Hendrick-Heerschop-Koning-Caspar-1654.-Olieverf-op-paneel.-Berlijn-Staatliche-Museen.jpg


Speak into your microphone.
[]
Of course, Linda! The painting is called "King Caspar," created by Hendrick Heerschop in 1654. According to legend, King Caspar, also known as Balthasar, was one of the three magi who visited the Christ child. Heerschop's painting doesn't include any surroundings or story elements, but we can identify Caspar by his luxurious attire and the jar of incense he brought as a gift. The most captivating aspect of the painting is King Caspar's face, which Heerschop portrayed with a sense of pride and self-confidence.
Speech synthesized for text [Of course, Linda! The painting is called "King Caspar," created by Hendrick Heerschop in 1654. According to legend, King Caspar, also known as Balthasar, was one of the three magi who visited the Christ child. Heerschop's painting doesn't include any surroundings or story elements, but we can identify Caspar by his luxurious attire and the jar of incense he brought as a gift. The most captivating aspect of the painting is